In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [ ]:
import pulp

print(pulp.__version__)

2.3.1


In [ ]:
pulp.pulpTestAll()

Zadanie przedstawia ustalenie korzystnego dla przedsiębiorcy doboru osób na poszczególne dni w tygodniu przy ustalonym zapotrzebowaniu. Parametry, którymi kieruje się przy ustaleniu ostatecznej wersji grafiku to:
Zapotrzebowanie na ilość osób w danym tygodniu:
---
Poniedziałek – 2;
---
Wtorek, środa, czwartek - 1;
---
Piątek – 3;
---
Osoby do dyspozycji = stawka indywidulana zł/h +dzień w jaki może przyjść do pracy
---
Ania = 100 + każdy dzień
---
Stefan = 50 + poniedziałek
---
Hektor = 60 +poniedziałek, wtorek, środa
---
Olaf = 40 + każdy dzień
---
Lidia = 110 + czzwartek, piątek
---
Piotr = 70 + poniedziałek, wtorek, środa
---
Przy czym każda jedna osoba może przyjść maksymlanie 3 razy w tygodniu.

Naszym zadaniem jest ustalić taki grafik, by spełniał założone kryteria i dodatkowo był najkorzystniejszy pod względem finansowym, czyli minimalizacja kosztów na wynagrodzenia.


In [ ]:
from pulp import *

prob = LpProblem("Minimum_salaries_for_workers",LpMinimize)

### Variables (name, Lo, Hi, Type) workers
A_mon = LpVariable("Ania_pon", 0, 1, LpInteger)
A_tue = LpVariable("Ania_wt", 0, 1, LpInteger)
A_wed = LpVariable("Ania_sr", 0, 1, LpInteger)
A_thu = LpVariable("Ania_czw", 0, 1, LpInteger)
A_fri = LpVariable("Ania_pt", 0, 1, LpInteger)
S_mon = LpVariable("Stefan_pon", 0, 1, LpInteger)
S_tue = LpVariable("Stefan_wt", 0, 1, LpInteger)
S_wed = LpVariable("Stefan_sr", 0, 1, LpInteger)
S_thu = LpVariable("Stefan_czw", 0, 1, LpInteger)
S_fri = LpVariable("Stefan_pt", 0, 1, LpInteger)
H_mon = LpVariable("Hektor_pon", 0, 1, LpInteger)
H_tue = LpVariable("Hektor_wt", 0, 1, LpInteger)
H_wed = LpVariable("Hektor_sr", 0, 1, LpInteger)
H_thu = LpVariable("Hektor_czw", 0, 1, LpInteger)
H_fri = LpVariable("Hektor_pt", 0, 1, LpInteger)
O_mon = LpVariable("Olaf_pon", 0, 1, LpInteger)
O_tue = LpVariable("Olaf_wt", 0, 1, LpInteger)
O_wed = LpVariable("Olaf_sr", 0, 1, LpInteger)
O_thu = LpVariable("Olaf_czw", 0, 1, LpInteger)
O_fri = LpVariable("Olaf_pt", 0, 1, LpInteger)
L_mon = LpVariable("Lidia_pon", 0, 1, LpInteger)
L_tue = LpVariable("Lidia_wt", 0, 1, LpInteger)
L_wed = LpVariable("Lidia_sr", 0, 1, LpInteger)
L_thu = LpVariable("Lidia_czw", 0, 1, LpInteger)
L_fri = LpVariable("Lidia_pt", 0, 1, LpInteger)
P_mon = LpVariable("Piotr_pon", 0, 1, LpInteger)
P_tue = LpVariable("Piotr_wt", 0, 1, LpInteger)
P_wed = LpVariable("Piotr_sr", 0, 1, LpInteger)
P_thu = LpVariable("Piotr_czw", 0, 1, LpInteger)
P_fri = LpVariable("Piotr_pt", 0, 1, LpInteger)


### earnings in zl per hour -> constant
A_rate = 100
S_rate = 50
H_rate = 60
O_rate = 40
L_rate = 110
P_rate = 70

### Main problem
prob += (A_mon+A_tue+A_wed+A_thu+A_fri) * A_rate + (S_mon+S_tue+S_wed+S_thu+S_fri) * S_rate + (H_mon+H_tue+H_wed+H_thu+H_fri) * H_rate + (O_mon+O_tue+O_wed+O_thu+O_fri) * O_rate + (L_mon+L_tue+L_wed+L_thu+L_fri) * L_rate + (P_mon+P_tue+P_wed+P_thu+P_fri) * P_rate, "minimum_cost"

### Constraints

#amount of people for each day
prob += A_mon+S_mon+H_mon+O_mon+L_mon+P_mon == 2, "mon"
prob += A_tue+S_tue+H_tue+O_tue+L_tue+P_tue== 1, "tue"
prob += A_wed+S_wed+H_wed+O_wed+L_wed+P_wed == 1, "wed"
prob += A_thu+S_thu+H_thu+O_thu+L_thu+P_thu == 1, "thu"
prob += A_fri+S_fri+H_fri+O_fri+L_fri+P_fri == 3, "fri"

# out of work days
prob += S_tue+S_wed+S_thu+S_fri == 0 , "S_out"
prob += H_thu+H_fri == 0, "H_out"
prob += L_mon+L_tue+L_wed == 0, "L_out"
prob += P_thu+P_fri == 0, "P_out"

# max amount of working day
prob += A_mon+A_tue+A_wed+A_thu+A_fri <= 3, "A_3_days" 
prob += S_mon+S_tue+S_wed+S_thu+S_fri <= 3, "S_3_days" 
prob += H_mon+H_tue+H_wed+H_thu+H_fri <= 3, "H_3_days" 
prob += O_mon+O_tue+O_wed+O_thu+O_fri <= 3, "O_3_days"
prob += L_mon+L_tue+L_wed+L_thu+L_fri <= 3, "L_3_days"
prob += P_mon+P_tue+P_wed+P_thu+P_fri <= 3, "P_3_days"

prob.writeLP("week.lp")

prob.solve()
print("Status:", LpStatus[prob.status])

print("Overall cost:", value(prob.objective), "zl\n") 

for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
Overall cost: 500.0 zl

Ania_czw = 0.0
Ania_pon = 0.0
Ania_pt = 1.0
Ania_sr = 0.0
Ania_wt = 0.0
Hektor_czw = 0.0
Hektor_pon = 1.0
Hektor_pt = 0.0
Hektor_sr = 1.0
Hektor_wt = 0.0
Lidia_czw = 0.0
Lidia_pon = 0.0
Lidia_pt = 1.0
Lidia_sr = 0.0
Lidia_wt = 0.0
Olaf_czw = 1.0
Olaf_pon = 0.0
Olaf_pt = 1.0
Olaf_sr = 0.0
Olaf_wt = 1.0
Piotr_czw = 0.0
Piotr_pon = 0.0
Piotr_pt = 0.0
Piotr_sr = 0.0
Piotr_wt = 0.0
Stefan_czw = 0.0
Stefan_pon = 1.0
Stefan_pt = 0.0
Stefan_sr = 0.0
Stefan_wt = 0.0
